### nearest neighbour heuristic 

In [16]:
#import libraries
import numpy as np
import pandas as pd

def read_excel_file(filename, sheet_name):
    """
    Read coordinates and demand values from a specific sheet in an Excel file.
    Assumes the data is in columns labeled 'X', 'Y', and 'Demand'.
    """
    df = pd.read_excel(filename, sheet_name=sheet_name)
    coordinates = df[['X', 'Y']].values
    demands = df['Demand'].values
    return coordinates, demands

In [17]:
def calculate_distance_matrix(coordinates):
    """
    Calculate the distance matrix between coordinates.
    """
    num_points = len(coordinates)
    dist_matrix = np.zeros((num_points, num_points))

    for i in range(num_points):
        for j in range(num_points):
            dist_matrix[i, j] = calculate_distance(coordinates, i, j)

    return dist_matrix

In [18]:
def calculate_distance(coordinates, i, j):
    """
    Calculate the Euclidean distance between two points.
    """
    x1, y1 = coordinates[i]
    x2, y2 = coordinates[j]
    return np.sqrt((x1 - x2) ** 2 + (y1 - y2) ** 2)


In [19]:
def calculate_total_distance(route, dist_matrix):
    """
    Calculate the total distance of a given route using the distance matrix.
    """
    total_distance = 0
    num_points = len(route)

    for i in range(num_points - 1):
        current_node = route[i]
        next_node = route[i + 1]
        total_distance += dist_matrix[current_node, next_node]

    return total_distance

In [20]:
def nearest_neighbor(dist_matrix, demands, capacity):
    """
    Apply the Nearest Neighbor heuristic to find initial routes for VRP.
    """
    num_points = dist_matrix.shape[0]
    visited = np.zeros(num_points, dtype=bool)  # Correctly define visited array here
    routes = []

    while np.sum(visited) < num_points:
        current_node = 0  # Start at node 0 (depot)
        
        # Ensure we don't start from an already visited node
        while visited[current_node]:
            current_node += 1
            if current_node >= num_points:
                break
        
        current_capacity = 0
        route = [current_node]
        visited[current_node] = True

        while True:
            current = route[-1]
            nearest = None
            min_dist = float('inf')

            for neighbor in range(num_points):
                if not visited[neighbor] and demands[neighbor] + current_capacity <= capacity:
                    if dist_matrix[current, neighbor] < min_dist:
                        nearest = neighbor
                        min_dist = dist_matrix[current, neighbor]

            if nearest is None:
                break

            route.append(nearest)
            visited[nearest] = True
            current_capacity += demands[nearest]

        routes.append(route)

    return routes


In [21]:
def format_output(routes):
    """
    Format the final routes as required.
    In this example, it returns a list of routes.
    """
    return routes


def vrp_solver(filename, sheet_name, capacity):
    """
    Solve the VRP using the provided filename for coordinates and vehicle capacity.
    """
    coordinates, demands = read_excel_file(filename, sheet_name)
    dist_matrix = calculate_distance_matrix(coordinates)
    routes = nearest_neighbor(dist_matrix, demands, capacity)
    formatted_routes = format_output(routes)
    return formatted_routes

In [23]:
#Use nearest neighbor
filename = r"C:/Users/user/Downloads/coordinates_demands_extended.xlsx" #Copy file path
sheet_name = "Sheet1"
capacity = 2010 # Specify the capacity of the vehicle
solution = vrp_solver(filename, sheet_name, capacity)

for route in solution:
    print(route)


[0, 19, 5, 26, 44, 36, 23, 17]
[1, 12, 25, 20, 30, 3, 38, 24, 13, 9]
[2, 41, 39, 47, 28, 35, 7, 49]
[4, 31, 37, 15, 46, 40, 48]
[6, 29, 21, 14, 32, 11]
[8, 27, 18, 16, 22]
[10, 42, 45, 43, 34]
[33]


### two - opt optimization 

In [28]:
# Two-opt optimization with validation to prevent ValueError
def two_opt(routes, dist_matrix, num_iterations):
    best_routes = routes.copy()

    for _ in range(num_iterations):
        selected_route_idx = np.random.randint(0, len(routes))
        selected_route = routes[selected_route_idx]

        # Skip if the route has fewer than 3 nodes
        if len(selected_route) < 3:
            continue

        # Randomly select two different indices for swapping
        i, j = np.random.randint(1, len(selected_route) - 1, size=2)

        if j < i:
            i, j = j, i

        new_route = selected_route.copy()
        new_route[i:j] = selected_route[j - 1: i - 1: -1]  # Reverse the path between i and j

        new_routes = routes.copy()
        new_routes[selected_route_idx] = new_route

        # Compare distances and update best route if improvement is found
        if calculate_total_distance(new_routes[selected_route_idx], dist_matrix) < calculate_total_distance(
                best_routes[selected_route_idx], dist_matrix):
            best_routes = new_routes

    return best_routes

# VRP solver with two-opt optimization
def vrp_solver2(filename, sheet_name, capacity, num_iterations):
    """
    Solve the VRP using the provided filename for coordinates, vehicle capacity,
    and number of iterations for the two-opt optimization.
    """
    coordinates, demands = read_excel_file(filename, sheet_name)
    dist_matrix = calculate_distance_matrix(coordinates)
    routes = nearest_neighbor(dist_matrix, demands, capacity)

    # Apply Two-opt optimization to each route
    for i in range(len(routes)):
        route = routes[i]
        
        # Preventing ValueError by skipping small routes
        if len(route) < 3:
            continue
        
        optimized_route = two_opt([route], dist_matrix, num_iterations)[0]
        routes[i] = optimized_route

    return routes

# File paths and parameters
filename = r"C:/Users/user/Downloads/coordinates_demands_extended.xlsx"
sheet_name = "Sheet1"  
capacity = 200
num_iterations = 100000

# Solve the VRP
solution = vrp_solver2(filename, sheet_name, capacity, num_iterations)

# Display the solution
print("\nOptimized Routes:")
for idx, route in enumerate(solution):
    print(f"Route {idx + 1}: {route}")



Optimized Routes:
Route 1: [0, 17, 3, 9]
Route 2: [1, 25]
Route 3: [2, 11]
Route 4: [4, 13, 49]
Route 5: [5, 40]
Route 6: [6, 48]
Route 7: [7]
Route 8: [8]
Route 9: [10]
Route 10: [12]
Route 11: [14]
Route 12: [15]
Route 13: [16]
Route 14: [18]
Route 15: [19]
Route 16: [20]
Route 17: [21]
Route 18: [22]
Route 19: [23]
Route 20: [24]
Route 21: [26]
Route 22: [27]
Route 23: [28]
Route 24: [29]
Route 25: [30]
Route 26: [31]
Route 27: [32]
Route 28: [33]
Route 29: [34]
Route 30: [35]
Route 31: [36]
Route 32: [37]
Route 33: [38]
Route 34: [39]
Route 35: [41]
Route 36: [42]
Route 37: [43]
Route 38: [44]
Route 39: [45]
Route 40: [46]
Route 41: [47]


In [ ]:
from scipy.optimize import linprog

# Coefficients of the objective function
# Minimize Z = 200x + 500y
c = [200, 500]

# Coefficients of the inequality constraints
# x + y <= 120
# 2x + 4y <= 400
A = [[1, 1], [2, 4]]
b = [120, 400]

# Bounds for x and y (non-negativity constraint)
x_bounds = (0, None)
y_bounds = (0, None)

# Solve the LP
result = linprog(c, A_ub=A, b_ub=b, bounds=[x_bounds, y_bounds], method='highs')

# Display the result
print("Optimal Solution:")
print("Sea Shipments (x):", result.x[0])
print("Air Shipments (y):", result.x[1])
print("Minimum Cost:", result.fun)


In [4]:
# Modify the constraints to avoid the zero solution
from scipy.optimize import linprog

# Coefficients of the objective function
# Minimize Z = 200x + 500y
c = [200, 500]

# New constraints
# Add a minimum shipment constraint: x + y >= 10
# Tighter capacity constraint: 2x + 4y <= 300

# Updating the coefficients and bounds
A = [[1, 1], [2, 4], [-1, -1]]  # Add the minimum shipment constraint as a negative inequality
b = [120, 300, -10]              # -10 represents x + y >= 10

# Solve the LP with modified constraints
result = linprog(c, A_ub=A, b_ub=b, bounds=[x_bounds, y_bounds], method='highs')

# Display the result
result.x[0], result.x[1], result.fun


(np.float64(10.0), np.float64(0.0), 2000.0)